In [21]:
import pandas as pd
import numpy as np
from copy import deepcopy
from tqdm import tqdm
import sys

sys.path.append('../')


from algorithms.dijkstra import CustomizedTimeDependentDijkstra, McCustomizedTimeDependentDijkstra
from algorithms.raptor import CustomizedRAPTOR, McCustomizedRAPTOR

from data_structures.graph import CustomizedTransportGraph
from data_structures.raptor_data_struct import CustomizedRAPTOR_DS
from data_structures.timetable import CustomizedTimetable
from algorithms.connection_scan_algorithm import CustomizedConnectionScanAlgorithm, McCustomizedConnectionScanAlgorithm

# Select city

In [2]:
CITY = 'kuopio'

# Build Transport Graphs

In [3]:
walk_connections_transitive = pd.read_csv(F'data/{CITY}/walk_connections_transitive.csv')

walk_connections = pd.read_csv(F'data/{CITY}/network_walk.csv', sep=';')
df_walk_invert = walk_connections.copy()
df_walk_invert = df_walk_invert.rename(columns={'from_stop_I': 'to_stop_I', 'to_stop_I': 'from_stop_I'})
walk_connections = pd.concat((walk_connections, df_walk_invert))

transport_connections = pd.read_csv(F'data/{CITY}/network_temporal_day.csv', sep=';')
transport_connections = transport_connections[
    transport_connections['from_stop_I']!=transport_connections['to_stop_I']
].reset_index(drop=True).sort_values(by=['dep_time_ut', 'arr_time_ut'])


In [4]:
tg = CustomizedTransportGraph(transport_connections=transport_connections, walk_connections=walk_connections)

# Precompute timetable nodes

### For standard graph

In [5]:
tg_cst = deepcopy(tg)
tg_bst = deepcopy(tg)
tg_ttn_fc_ascending = deepcopy(tg)
tg_ttn_fc_descending = deepcopy(tg)

tg_cst.optimize_binary_search(method='cst')
tg_bst.optimize_binary_search(method='bst')

tg_ttn_fc_ascending.fractional_cascading_precomputation('ascending')
tg_ttn_fc_descending.fractional_cascading_precomputation('descending')

100%|████████████████████████████████████████████████████████████████| 548/548 [00:00<00:00, 11696.38it/s]


# Prepare data for Raptor

In [6]:
raptor_ds = CustomizedRAPTOR_DS(transport_connections=transport_connections, walk_connections=walk_connections_transitive)

# Prepare data for Connection Scan Algorithm

In [7]:
timetable = CustomizedTimetable(transport_connections=transport_connections, walk_connections=walk_connections_transitive)

# Compare solutions

In [19]:
N = 1000
algorithms = [ 'tg', 'tg_bst', 'tg_cst', 'tg_ttn_fc_ascending', 'tg_ttn_fc_descending', 'csa', 'raptor']
results = []


test_data = pd.read_csv(F'graph/{CITY}/{CITY}_test_data.pkl')
        
duration = {x: [] for x in algorithms}
explored_nodes = {x: [] for x in algorithms}
trip_duration = {x: [] for x in algorithms}

for index, row in tqdm(test_data.iterrows(), total=N): 
    for algorithm in algorithms:
        if algorithm == 'tg':
            pathfinding = CustomizedTimeDependentDijkstra(graph=tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'],
                                                          walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path(60)
            duration[algorithm].append(path['duration'])
            trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'tg_bst':
            pathfinding = CustomizedTimeDependentDijkstra(graph=tg_bst,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'],
                                                          walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path(60, 
                                             time_table_nodes='bst')
            duration[algorithm].append(path['duration'])
            trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'tg_cst':
            pathfinding = CustomizedTimeDependentDijkstra(graph=tg_cst,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'],
                                                          walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path(60, 
                                             time_table_nodes='cst')
            duration[algorithm].append(path['duration'])
            trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'tg_ttn_fc_ascending':
            pathfinding = CustomizedTimeDependentDijkstra(graph=tg_ttn_fc_ascending,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'],
                                                          walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path(60, 
                                             time_table_nodes='fc')
            duration[algorithm].append(path['duration'])
            
            trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'tg_ttn_fc_descending':
            pathfinding = CustomizedTimeDependentDijkstra(graph=tg_ttn_fc_descending,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'],
                                                          walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path(60, 
                                             time_table_nodes='fc')
            duration[algorithm].append(path['duration'])
            
            trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'csa':
            pathfinding = CustomizedConnectionScanAlgorithm(graph=timetable,
                  start_time=row['start_time'],
                  start_node=row['start_node'], 
                  end_node=row['end_node'],
                                                          walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path()
            duration[algorithm].append(path['duration'])
                
            trip_duration[algorithm].append(path['arrival'] - row['start_time'])

        elif algorithm == 'raptor':
            pathfinding = CustomizedRAPTOR(graph=raptor_ds,
                  start_time=row['start_time'],
                  start_node=row['start_node'], 
                  end_node=row['end_node'],
                                           walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path()
            duration[algorithm].append(path['duration'])
                
            trip_duration[algorithm].append(path['arrival'] - row['start_time'])

100%|█████████████████████████████████████████████████████████████████| 1000/1000 [02:03<00:00,  8.07it/s]


# Results

In [20]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.std(duration[algorithm]))

tg 6.157 3.9350160101326144
tg_bst 6.438 9.264888342554377
tg_cst 5.658 3.6990587992082524
tg_ttn_fc_ascending 5.512 3.638386455559662
tg_ttn_fc_descending 5.54 3.6840738320506015
csa 18.149 13.255896763327632
raptor 72.189 40.04286801666434


# Mult criteria search

In [24]:
N = 1000
algorithms = [ 'tg', 'tg_bst', 'tg_cst', 'tg_ttn_fc_ascending', 'tg_ttn_fc_descending', 'csa', 'raptor']
results = []


test_data = pd.read_csv(F'graph/{CITY}/{CITY}_test_data.pkl')
        
duration = {x: [] for x in algorithms}
explored_nodes = {x: [] for x in algorithms}
trip_duration = {x: [] for x in algorithms}

for index, row in tqdm(test_data.iterrows(), total=N): 
    for algorithm in algorithms:
        if algorithm == 'tg':
            pathfinding = McCustomizedTimeDependentDijkstra(graph=tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'],
                                                          walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path()
            duration[algorithm].append(path['duration'])
        elif algorithm == 'tg_bst':
            pathfinding = McCustomizedTimeDependentDijkstra(graph=tg_bst,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'],
                                                          walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path(time_table_nodes='bst')
            duration[algorithm].append(path['duration'])
        elif algorithm == 'tg_cst':
            pathfinding = McCustomizedTimeDependentDijkstra(graph=tg_cst,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'],
                                                          walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path(time_table_nodes='cst')
            duration[algorithm].append(path['duration'])
        elif algorithm == 'tg_ttn_fc_ascending':
            pathfinding = McCustomizedTimeDependentDijkstra(graph=tg_ttn_fc_ascending,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'],
                                                          walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path(time_table_nodes='fc')
            duration[algorithm].append(path['duration'])
        elif algorithm == 'tg_ttn_fc_descending':
            pathfinding = McCustomizedTimeDependentDijkstra(graph=tg_ttn_fc_descending,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'],
                                                          walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path(time_table_nodes='fc')
            duration[algorithm].append(path['duration'])
        elif algorithm == 'csa':
            pathfinding = McCustomizedConnectionScanAlgorithm(graph=timetable,
                  start_time=row['start_time'],
                  start_node=row['start_node'], 
                  end_node=row['end_node'],
                                                          walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path()
            duration[algorithm].append(path['duration'])
                

        elif algorithm == 'raptor':
            pathfinding = McCustomizedRAPTOR(graph=raptor_ds,
                  start_time=row['start_time'],
                  start_node=row['start_node'], 
                  end_node=row['end_node'],
                                           walking_speed = row['walking_speed'])
            path = pathfinding.shortest_path()
            duration[algorithm].append(path['duration'])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:18:01<00:00,  4.68s/it]


# Results

In [25]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.std(duration[algorithm]))

tg 121.616 61.235990593767646
tg_bst 113.743 58.31451749778952
tg_cst 111.838 57.400346305575546
tg_ttn_fc_ascending 103.643 53.693682598607445
tg_ttn_fc_descending 103.432 54.243998525182484
csa 744.626 432.2863866975226
raptor 2195.996 1608.2302372434117
